# Lab 1: Building own n-gram language model

In this lab, build a language model based on n-grams and the Maximum Likelihood Estimation.

Assume that the text is already "tokenized" (split up into words). We'll cover this process in more depth in the next module.

As an example, let's work with two sentences from "[The Disappearance of Lady Frances Carfax](https://en.wikipedia.org/wiki/The_Disappearance_of_Lady_Frances_Carfax)", a short story written by [Sir Arthur Conan Doyle](https://en.wikipedia.org/wiki/Arthur_Conan_Doyle).

In [8]:
# Tokens for the sentence "It shows, my dear Watson, that we are dealing
# with an exceptionally astude and dangerous man."
sample1 = ['It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that',
           'we', 'are', 'dealing', 'with', 'an', 'exceptionally',
           'astute', 'and', 'dangerous', 'man', '.']
# Tokens for the sentence "How would Lausanne do, my dear Watson?"
sample2 = ['How', 'would', 'Lausanne', 'do', ',', 'my', 'dear',
           'Watson', '?']

Your first task is to write a function that splits the `tokens` sequence
into its `n`-grams.

For instance, when `tokens=sample1` and `n=3`, your function should
return:

```python
[('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.')]
```
 
Note: You should return a python [`list`](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) containing [`tuple`](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)s. `tuple`s are immutable sequences, which will be useful later on when you build your language model.

In [9]:
from typing import List, Tuple
def build_ngrams(tokens: List[str], n: int) -> List[Tuple[str]]:
    result =[]
    for i in range(len(tokens)-n+1):
        tmp = tokens[i: i+n]
        result.append(tuple(tmp))
    return result
# Example:
print('ln: ', len(build_ngrams(sample2, n=2)))
build_ngrams(sample2, n=2)

ln:  8


[('How', 'would'),
 ('would', 'Lausanne'),
 ('Lausanne', 'do'),
 ('do', ','),
 (',', 'my'),
 ('my', 'dear'),
 ('dear', 'Watson'),
 ('Watson', '?')]

With the current function, there's no way to know whether an n-gram is at the beginning, middle, or end of the sequence. To overcome this problem, n-gram language models often include special "beginning-of-string" (BOS) and "end-of-string" (EOS) control tokens.

Write a new version of your `build_ngrams` function that includes these control tokens. For instance, when `tokens=sample1` and `n=3`, your new function should return:

```python
[('<BOS>', '<BOS>', 'It'),
 ('<BOS>', 'It', 'shows'),
 ('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.'),
 ('man', '.', '<EOS>'),
 ('.', '<EOS>', '<EOS>')]
```

In [10]:
BOS = '<BOS>'
EOS = '<EOS>'
def build_ngrams_ctrl(tokens: List[str], n: int) -> List[Tuple[str]]:
    tokens_copy = tokens.copy()

    lenn = len(tokens_copy)
    for i in range(n-1):
        tokens_copy.insert(i, BOS)
    new_end = lenn+n-1
    for j in range(new_end, new_end+n-1):
        tokens_copy.insert(j, EOS)
#     tokens_copy.insert(lenn+3, EOS)
    print('tokens : ', tokens_copy)
    result = build_ngrams(tokens_copy, n)
    return result
    # Example:
res = build_ngrams_ctrl(sample1, n=2)
print('final len: ', len(res))
res

tokens :  ['<BOS>', 'It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that', 'we', 'are', 'dealing', 'with', 'an', 'exceptionally', 'astute', 'and', 'dangerous', 'man', '.', '<EOS>']
final len:  20


[('<BOS>', 'It'),
 ('It', 'shows'),
 ('shows', ','),
 (',', 'my'),
 ('my', 'dear'),
 ('dear', 'Watson'),
 ('Watson', ','),
 (',', 'that'),
 ('that', 'we'),
 ('we', 'are'),
 ('are', 'dealing'),
 ('dealing', 'with'),
 ('with', 'an'),
 ('an', 'exceptionally'),
 ('exceptionally', 'astute'),
 ('astute', 'and'),
 ('and', 'dangerous'),
 ('dangerous', 'man'),
 ('man', '.'),
 ('.', '<EOS>')]

Now that you can build n-grams, we have almost everything we need to build an n-gram language model.

To compute Maximum Likelihood Estimations, you first need to count the number of times each word follows an n-gram of size `n-1`. You can build this structure as a Python [`dict`](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that maps the n-grams of size `n-1` to another `dict` that maps the following words to their respective counts.

For instance, when `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
    ('<BOS>', 'It'): {'shows': 1},
    ('<BOS>', 'How'): {'would': 1},
    ...
    ('my', 'dear'): {'Watson': 2},
    ('dear', 'Watson'): {',': 1, '?': 1},
    ...
}
```

In [11]:
from typing import Dict
def count_ngrams(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, int]]:
    result = {}
    for sample in texts:
        res = build_ngrams_ctrl(sample, n)
        for subtext in res:
            preseq = subtext[0:n-1]
            if preseq not in result:
                next_word_count = {}
                next_word_count[subtext[n-1]] = 1
                result[preseq] = next_word_count
            else:
                next_word_count = result[preseq]
                if subtext[n-1] in next_word_count:
                    next_word_count[subtext[n-1]] += 1
                else:
                    next_word_count[subtext[n-1]] = 1
                result[preseq] = next_word_count
    return result 

# Example:
count_ngrams([sample1, sample2], n=3)

tokens :  ['<BOS>', '<BOS>', 'It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that', 'we', 'are', 'dealing', 'with', 'an', 'exceptionally', 'astute', 'and', 'dangerous', 'man', '.', '<EOS>', '<EOS>']
tokens :  ['<BOS>', '<BOS>', 'How', 'would', 'Lausanne', 'do', ',', 'my', 'dear', 'Watson', '?', '<EOS>', '<EOS>']


{('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
 ('<BOS>', 'It'): {'shows': 1},
 ('It', 'shows'): {',': 1},
 ('shows', ','): {'my': 1},
 (',', 'my'): {'dear': 2},
 ('my', 'dear'): {'Watson': 2},
 ('dear', 'Watson'): {',': 1, '?': 1},
 ('Watson', ','): {'that': 1},
 (',', 'that'): {'we': 1},
 ('that', 'we'): {'are': 1},
 ('we', 'are'): {'dealing': 1},
 ('are', 'dealing'): {'with': 1},
 ('dealing', 'with'): {'an': 1},
 ('with', 'an'): {'exceptionally': 1},
 ('an', 'exceptionally'): {'astute': 1},
 ('exceptionally', 'astute'): {'and': 1},
 ('astute', 'and'): {'dangerous': 1},
 ('and', 'dangerous'): {'man': 1},
 ('dangerous', 'man'): {'.': 1},
 ('man', '.'): {'<EOS>': 1},
 ('.', '<EOS>'): {'<EOS>': 1},
 ('<BOS>', 'How'): {'would': 1},
 ('How', 'would'): {'Lausanne': 1},
 ('would', 'Lausanne'): {'do': 1},
 ('Lausanne', 'do'): {',': 1},
 ('do', ','): {'my': 1},
 ('Watson', '?'): {'<EOS>': 1},
 ('?', '<EOS>'): {'<EOS>': 1}}

You're almost there! The last step is to convert the counts into probability estimates.

When `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
    ('<BOS>', 'It'): {'shows': 1.0},
    ('<BOS>', 'How'): {'would': 1.0},
    ...
    ('my', 'dear'): {'Watson': 1.0},
    ('dear', 'Watson'): {',': 0.5, '?': 0.5},
    ...
}
```

In [12]:
from typing import Dict
def build_ngram_model(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, float]]:
    count_ngram_dict = count_ngrams(texts, n)
    ngram_result = {}
    for k,v in count_ngram_dict.items():
        total_sum = sum(v.values())
        res = {key: val/total_sum for key,val in v.items()}
        ngram_result[k] = res
    return ngram_result

# Example:
build_ngram_model([sample1, sample2], n=3)

tokens :  ['<BOS>', '<BOS>', 'It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that', 'we', 'are', 'dealing', 'with', 'an', 'exceptionally', 'astute', 'and', 'dangerous', 'man', '.', '<EOS>', '<EOS>']
tokens :  ['<BOS>', '<BOS>', 'How', 'would', 'Lausanne', 'do', ',', 'my', 'dear', 'Watson', '?', '<EOS>', '<EOS>']


{('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
 ('<BOS>', 'It'): {'shows': 1.0},
 ('It', 'shows'): {',': 1.0},
 ('shows', ','): {'my': 1.0},
 (',', 'my'): {'dear': 1.0},
 ('my', 'dear'): {'Watson': 1.0},
 ('dear', 'Watson'): {',': 0.5, '?': 0.5},
 ('Watson', ','): {'that': 1.0},
 (',', 'that'): {'we': 1.0},
 ('that', 'we'): {'are': 1.0},
 ('we', 'are'): {'dealing': 1.0},
 ('are', 'dealing'): {'with': 1.0},
 ('dealing', 'with'): {'an': 1.0},
 ('with', 'an'): {'exceptionally': 1.0},
 ('an', 'exceptionally'): {'astute': 1.0},
 ('exceptionally', 'astute'): {'and': 1.0},
 ('astute', 'and'): {'dangerous': 1.0},
 ('and', 'dangerous'): {'man': 1.0},
 ('dangerous', 'man'): {'.': 1.0},
 ('man', '.'): {'<EOS>': 1.0},
 ('.', '<EOS>'): {'<EOS>': 1.0},
 ('<BOS>', 'How'): {'would': 1.0},
 ('How', 'would'): {'Lausanne': 1.0},
 ('would', 'Lausanne'): {'do': 1.0},
 ('Lausanne', 'do'): {',': 1.0},
 ('do', ','): {'my': 1.0},
 ('Watson', '?'): {'<EOS>': 1.0},
 ('?', '<EOS>'): {'<EOS>': 1.0}}

A language model built from only a few sentences is not very informative. Let's scale up and see what your language model looks like when we train on the complete works of Sir Arthur Conon Doyle!

In [13]:
full_text = []
with open('arthur-conan-doyle.tok.train.txt', 'rt') as fin:
    for line in fin:
        full_text.append(list(line.split()))
model = build_ngram_model(full_text, n=3)

FileNotFoundError: [Errno 2] No such file or directory: 'arthur-conan-doyle.tok.train.txt'

In [14]:
for prefix in [(BOS, BOS), (BOS, 'It'), ('It', 'was'), ('my', 'dear')]:
    print(*prefix)
    sorted_probs = sorted(model[prefix].items(), key=lambda x: -x[1])
    for k, v in sorted_probs[:5]:
        print(f'\t{k}\t{v:.4f}')
    print(f'\t[{len(sorted_probs)-5} more...]')

<BOS> <BOS>


NameError: name 'model' is not defined